## Project notebook

Using tensorflow and keras wrapper

In [1]:
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
import utils
import models
import h5py
from PIL import Image

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (20.0, 16.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import merge
from keras.optimizers import SGD, Adadelta, Adam, RMSprop
from keras.utils import np_utils
from keras.models import load_model

Using TensorFlow backend.


## Load training data

In [2]:
X_train, y_train, meanImage, labels = utils.load_training_data();

num_classes = len(labels);
crop = 4; 
X_train = utils.crop_image(X_train, crop, crop);
print("cropped X_train shape:", X_train.shape)
print("number of classes = ", num_classes)

KeyboardInterrupt: 

In [ ]:
# Visualize some examples from the dataset.
# We show a few examples of training images from each class.
if crop != 0:
    meanImageCropped = meanImage[crop:-crop, crop:-crop, :];
else:
    meanImageCropped = meanImage;    
    
classes = labels[0:10];
num_classes = len(classes);
samples_per_class = 7;
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow((128*X_train[idx] + meanImageCropped).astype('uint8'), interpolation="nearest")
        plt.axis('off')
        if i == 0:
            plt.title(cls)

# mean image
plt.figure(figsize = (5,4));
plt.imshow(meanImage.astype('uint8'), interpolation="nearest")
plt.title('Mean image')
plt.axis('off')

## Load validation data

In [ ]:
X_val, y_val = utils.load_validation_data(labels, meanImage);
X_val = utils.crop_image(X_val, crop, crop);
print("cropped X_val shape:", X_val.shape)

In [ ]:
# Visualize some examples from the dataset.
# We show a few examples of training images from each class.
plt.figure();
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_val == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow((128*X_val[idx] + meanImageCropped).astype('uint8'), interpolation="nearest")
        plt.axis('off')
        if i == 0:
            plt.title(cls)


## Train and evaluate model

In [ ]:
# Load particular model saved
# model = load_model('milestone_model.h5')

In [ ]:
# From https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3
# model = models.VGG_16((64-2*crop, 64-2*crop, 3));
#model = models.CNN_BN((64-2*crop, 64-2*crop, 3), drop_rate=0.75);
#model = models.CNN64_BN(input_shape=(64, 64, 3), drop_rate=0.7);
model = models.cnn_model3(input_shape=(64-2*crop, 64-2*crop, 3), drop_rate=0.5);

#opt = SGD(lr=0.03, decay=1e-4, momentum=0.9, nesterov=True)
#opt = RMSprop(lr=0.0005)
opt = Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

y_train_categorical = np_utils.to_categorical(y_train, num_classes=200);
y_val_categorical = np_utils.to_categorical(y_val, num_classes=200);

history = model.fit(X_train, y_train_categorical, batch_size=200, epochs=20, validation_data=(X_val, y_val_categorical), verbose=True)
metrics = model.evaluate(X_val, y_val_categorical, verbose=True)


## Transfer learning

In [ ]:
model_name = 'modified_VGG16'
weights_path = 'vgg16_weights_tf_dim_ordering_tf_kernels.h5' 

# Show weights in file
f = h5py.File(weights_path)
print([key for key in f.keys()]);

# Create model
model = models.modified_VGG16(input_shape=(64-2*crop, 64-2*crop, 3), drop_rate=0.5, stddev=1e-1);
       
# Load model weights from file. Only weights with existing names are loaded
model.load_weights(weights_path, by_name=True)

# Only train fully connected layers
for i, layer in enumerate(model.layers):
    if layer.__class__.__name__ != 'Dense':
        model.layers[i].trainable = False;
    
# Show summary
model.summary()

# Train and evaluate
opt = Adam(lr=5e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

y_train_categorical = np_utils.to_categorical(y_train, num_classes=200);
y_val_categorical = np_utils.to_categorical(y_val, num_classes=200);

history = model.fit(X_train, y_train_categorical, batch_size=200, epochs=10, validation_data=(X_val, y_val_categorical), verbose=True)
metrics = model.evaluate(X_val, y_val_categorical, verbose=True)

In [ ]:
from pylab import savefig

print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.ylabel('Accuracy', fontsize=20)
plt.xlabel('epoch', fontsize=20)
plt.legend(['Training', 'Validation'], loc='upper left')
plt.show()
savefig(model_name + '_accuracy.png', bbox_inches='tight')

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('Loss', fontsize=20)
plt.xlabel('epoch', fontsize=20)
plt.legend(['Training', 'Validation'], loc='upper left')
plt.show()
savefig(model_name + '_loss.png', bbox_inches='tight')

model.save(model_name + '_weights.h5')

utils.save_historty(history.historty, model_name + '_history.txt')


## Run on test set

In [ ]:
# Load test data
X_test, test_files = utils.load_test_data(meanImage);
X_test = utils.crop_image(X_test, crop, crop);
print("cropped X_test shape:", X_test.shape)

# Visualize some examples from the dataset.
# We show a few examples of training images from each class.
plt.figure();
num_samples = 10;
idxs = np.random.choice(range(len(test_files)), num_samples, replace=False)
for i, idx in enumerate(idxs):
    plt.subplot(1, num_samples, i+1)
    plt.imshow((128*X_test[idx] + meanImageCropped).astype('uint8'), interpolation="nearest")
    plt.axis('off')
    plt.title(test_files[idx])

# Make predictions on test set
probs = model.predict(X_test, batch_size=32, verbose=0);
predictions = np.argmax(probs, axis=1);
print(predictions.shape)

prediceted_classes = [];
for y in predictions:
    prediceted_classes.append(ids[y]);

# Save predictions
utils.save_test_predictions(model_name + '_test_predictions.txt', test_files, prediceted_classes)